# Parameters and Loss Visualization in Tensorflow

Note: Naming variables, ops, etc is also useful for debugging.

In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

## Load MNIST dataset

In [2]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Parameters

In [5]:
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1

## tf Graph input

In [6]:
x = tf.placeholder(tf.float32, [None, 784], name='x') # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10], name='y') # 0-9 digits recognition => 10 classes

## Define the model

Here we add layer name scopes for better graph visualization

In [7]:
def model(_x, _weights, _biases, p_keep_input, p_keep_hidden):
    _x = tf.nn.dropout(_x, p_keep_input)
    with tf.name_scope('layer1'):
        layer1 = tf.nn.relu(tf.nn.bias_add(tf.matmul(_x, _weights['layer1']), _biases['layer1']))
        layer1 = tf.nn.dropout(layer1, p_keep_hidden)
    with tf.name_scope('layer2'):
        layer2 = tf.nn.relu(tf.nn.bias_add(tf.matmul(layer1, _weights['layer2']), _biases['layer2']))
        layer2 = tf.nn.dropout(layer2, p_keep_hidden)
    with tf.name_scope('out'):
        out = tf.nn.bias_add(tf.matmul(layer2, _weights['out']), _biases['out'])
    
    return out

## Set model weights and bias

In [8]:
def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.1), name=name)

In [7]:
def init_biases(shape, name):
    return tf.Variable(tf.zeros(shape), name=name)

In [8]:
weights = {
    'layer1': init_weights([784, 625], 'layer1'),
    'layer2': init_weights([625, 625], 'layer1'),
    'out': init_weights([625, 10], 'out')
}

In [9]:
biases = {
    'layer1': init_biases([625], 'layer1'),
    'layer2': init_biases([625], 'layer2'),
    'out': init_biases([10], 'out')
}

### Add histogram summaries for weights

In [10]:
tf.histogram_summary('layer1', weights['layer1'])
tf.histogram_summary('layer2', weights['layer2'])
tf.histogram_summary('out', weights['out'])

<tf.Tensor 'HistogramSummary_2:0' shape=() dtype=string>

## Define dropout variables

In [11]:
dropout_input = tf.placeholder(tf.float32, name='dropout_input')
dropout_hidden = tf.placeholder(tf.float32, name='dropout_hidden')

## Construct the model

In [12]:
pred = model(x, weights, biases, dropout_input, dropout_hidden)

## Minimize error using cross-entropy

In [13]:
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
    tf.scalar_summary('cost', cost)

## Calculate accuracy

In [14]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    tf.scalar_summary('accuracy', accuracy)

## Initializing the variables

In [15]:
init = tf.initialize_all_variables()

## Merge all summaries to a single operator

In [16]:
merged_summary_op = tf.merge_all_summaries()

## Launch the graph

In [17]:
with tf.Session() as sess:
    sess.run(init)
    
    # Set logs writer into folder /tmp/tensorflow_logs
    summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', graph=sess.graph)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys,
                                           dropout_input: 0.8, dropout_hidden: 0.5})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys,
                                                  dropout_input: 1.0, dropout_hidden: 1.0})/total_batch
            # Write logs at every iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys,
                                                                 dropout_input: 1.0, dropout_hidden: 1.0})
            summary_writer.add_summary(summary_str, epoch*total_batch + i)
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch: {:4d}, cost= {:.9f}".format(epoch+1, avg_cost))
            
    print("Optimization Finished!")
    
    # Test model
    print("Accuracy: {}".format(accuracy.eval({x: mnist.test.images, y: mnist.test.labels,
                                               dropout_input: 1.0, dropout_hidden: 1.0})))

Epoch:    1, cost= 0.669172617
Epoch:    2, cost= 0.418671131
Epoch:    3, cost= 0.370321383
Epoch:    4, cost= 0.342486015
Epoch:    5, cost= 0.320802517
Epoch:    6, cost= 0.302726998
Epoch:    7, cost= 0.288104926
Epoch:    8, cost= 0.276460817
Epoch:    9, cost= 0.265696452
Epoch:   10, cost= 0.255671011
Optimization Finished!
Accuracy: 0.9320999979972839


## Run in the command line
    tensorboard --logdir=/tmp/tensorflow_logs